In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf


In [2]:
data = np.array(pd.read_csv("Dataset/Text_Emotion_Data.csv"))
stopwords = np.array(pd.read_csv("Dataset/stopwords.txt", sep=" ", header=None)).flatten()

In [3]:
X = data[:, 0]
y = data[:, 1]

In [4]:
import re

def remove_none_alpha(x):
    regex = re.compile('[^a-zA-Z]')
    return regex.sub(' ', x)

In [5]:
stopwords = [remove_none_alpha(w) for w in stopwords]

In [6]:
X_seq = []
for sentence in X:
    words_seq = remove_none_alpha(sentence).lower().split(" ")
    words_seq = [w for w in words_seq if len(w) > 2]
    words_seq = [w for w in words_seq if w not in stopwords]
    X_seq.append(words_seq)

In [7]:
max_len = len(max(X_seq, key=len))

# create a new list of lists with the desired size, padding with empty strings
sequences = [['' for _ in range(max_len)] for _ in range(len(X_seq))]

# copy the elements from the original lists into the new list, padding with empty strings as needed
for i, sequence in enumerate(X_seq):
    sequences[i][:len(sequence)] = sequence

In [8]:
def one_hot_encode(sequence, word_to_idx):
    num_words = len(word_to_idx)
    encoding = np.zeros((len(sequence), num_words))
    for i, word in enumerate(sequence):
        if word in word_to_idx:
            encoding[i, word_to_idx[word]] = 1
    return encoding

unique_words = list(np.unique([word for sequence in sequences for word in sequence]))
word_to_idx = {word: idx for idx, word in enumerate(unique_words)}

encoded_sequences = []
for sequence in sequences:
    encoded_sequences.append(one_hot_encode(sequence, word_to_idx))

encoded_sequences = np.array(encoded_sequences)

In [9]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
y_encoded = le.fit_transform(y)
y_encoded_onehot = tf.keras.utils.to_categorical(y_encoded, num_classes = 5)

In [10]:
def custom_train_test_split(x, y, _y):
    unique_classes = np.unique(_y)
    train_indices = []
    test_indices = []
    
    for cls in unique_classes:
        cls_indices = np.where(_y == cls)[0]
        train_indices.extend(cls_indices[:-150])
        test_indices.extend(cls_indices[-150:])
    
    train_x = x[train_indices]
    train_y = y[train_indices]
    test_x = x[test_indices]
    test_y = y[test_indices]
    
    return train_x, train_y, test_x, test_y

In [11]:
X_train, y_train, X_test, y_test = custom_train_test_split(encoded_sequences, \
                                                            y_encoded_onehot, y_encoded)

In [13]:
import tensorflow as tf

input_dim = X_train.shape[2]
hidden_dim = 100
output_dim = 5
num_steps = X_train.shape[1]

inputs = tf.keras.layers.Input(shape=(num_steps, input_dim))
hidden_layer = tf.keras.layers.SimpleRNN(hidden_dim, activation='tanh')(inputs)
outputs = tf.keras.layers.Dense(output_dim, activation='relu')(hidden_layer)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

# optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
# optimizer = tf.keras.optimizers.Adadelta(learning_rate=0.01)
# alternate losses: sparse_categorical_crossentropy, categorical_crossentropy
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

# early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=31, validation_data=(X_test, y_test))


Epoch 1/20
81/81 [==============================] - 13s 139ms/step - loss: 0.1652 - accuracy: 0.2432 - val_loss: 0.1504 - val_accuracy: 0.3307
Epoch 2/20
81/81 [==============================] - 11s 132ms/step - loss: 0.0538 - accuracy: 0.8884 - val_loss: 0.1374 - val_accuracy: 0.4933
Epoch 3/20
81/81 [==============================] - 10s 125ms/step - loss: 0.0216 - accuracy: 0.9696 - val_loss: 0.1259 - val_accuracy: 0.5440
Epoch 4/20
81/81 [==============================] - 10s 128ms/step - loss: 0.0083 - accuracy: 0.9920 - val_loss: 0.1225 - val_accuracy: 0.5453
Epoch 5/20
81/81 [==============================] - 10s 129ms/step - loss: 0.0039 - accuracy: 0.9972 - val_loss: 0.1184 - val_accuracy: 0.5720
Epoch 6/20
81/81 [==============================] - 11s 132ms/step - loss: 0.0022 - accuracy: 0.9976 - val_loss: 0.1168 - val_accuracy: 0.5720
Epoch 7/20
81/81 [==============================] - 11s 131ms/step - loss: 0.0014 - accuracy: 0.9976 - val_loss: 0.1139 - val_accuracy: 0.5960

In [ ]:
# input_dim = X_train.shape[2]
# hidden_dim = 60
# output_dim = 5
# num_steps = X_train.shape[1]

# model = tf.keras.Sequential()

# model.add(tf.keras.layers.SimpleRNN(hidden_dim, activation='tanh', input_shape=(num_steps, input_dim)))
# model.add(tf.keras.layers.Dense(output_dim, activation='softmax'))

# model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.summary()

# model.fit(X_train, y_train, epochs=10, batch_size=31, validation_data=(X_test, y_test))

In [15]:
loss, acc = model.evaluate(X_test, y_test)

24/24 [==============================] - 3s 102ms/step - loss: 0.1097 - accuracy: 0.6027


In [16]:
import pickle
import datetime

today = datetime.datetime.today()

with open(f'./models/{int(acc * 100)}-model-({today.day}-{today.hour}-{today.minute}).p', 'wb') as outfile:
    pickle.dump(model, outfile)